# Hybrid DFT data processing
This notebook handles the raw DFT data and produces various processed datasets to be read in by the main notebook.


#### Requirements 
- The data must be downloaded from [ZENODO LINK] and put into the `raw_data/` directory in the parent folder.
- [PolaronMobility.jl](https://github.com/jarvist/PolaronMobility.jl) and any dependencies.
- [Phonopy-spectroscopy](https://github.com/JMSkelton/Phonopy-Spectroscopy) and any dependencies (for the vasp-ir script).   
`pip install numpy scipy pandas pymatgen cmake BoltzTrap2` 



#### Steps
- Get high frequency dielectric response (eps_inf) from VASP LOPTICS calculation
- Get ionic contribution to dielectric response (eps_ionic) from VASP DFPT calculation
- Get effective masses from VASP LOPTICS calculation using BoltzTraP
- Calculate E_polaron using the same approach as in the main notebook
- write to `hybrid_calcs_data.json`
- Calculate simulated IR absorption spectra (calling vasp-ir script)
- Calculate characteristic phonon frequencies (calling characteristic_phonon_Freq.jl julia script)
- Calculate mobilities using PolaronMobility.jl
- write to `hybrid_calcs_data_and_mobilities.json`

In [1]:
### Imports
from pymatgen.io.vasp import Vasprun, Outcar
import os
import subprocess
import warnings
import numpy as np
from scipy.stats import hmean   # harmonic mean
from pymatgen.electronic_structure.boltztrap import BoltztrapRunner, BoltztrapAnalyzer
from scipy.stats import hmean
from pandas import DataFrame
import pandas as pd

In [2]:
warnings.filterwarnings('ignore')
data_dicts = []

NB: most of the LOPTICS calcs were done in two stages - for bigger calculations we obtain a DOS using the tetrahedron method and then do an LOPTICS run afterwards, reading in the WAVECAR and setting ALGO to none. This is because doing a single run tends to max out the memory and kill the calculation. For this reason, we filter warnings about unconverged calculations. The raw data for the tetrahedron calculations are also available in the `hybrid_calcs_raw` directory. 

In [3]:
# Extract eps_inf 
for root, dirs, files in os.walk('../raw_data/hybrid_calcs_raw/'):
    if root.endswith('loptics/loptics'):
        for name in files:
            if name == 'vasprun.xml':
                vasprun = Vasprun(os.path.join(root,name))
                form = vasprun.structures[0].composition.reduced_formula
                eps_inf = vasprun.dielectric_data['density'][1][0]
                eps_inf_poly = np.mean(eps_inf[:3])
                print(form, 'eps_inf = {}'.format(eps_inf_poly))
                data_dicts.append({'formula': form, 'eps_inf': eps_inf_poly})

NaSbO3 eps_inf = 2.784433333333333
NaAg3O2 eps_inf = 4.383966666666667
PtO2 eps_inf = 7.0860666666666665
CuRhO2 eps_inf = 8.2478
LiAg3O2 eps_inf = 4.962
K2TiO3 eps_inf = 2.9921333333333333
Ca4Bi2O eps_inf = 9.9097
LiNbO2 eps_inf = 7.705966666666666
YZnAsO eps_inf = 8.693433333333333
NaNbO2 eps_inf = 7.3290999999999995
YZnPO eps_inf = 7.1369
LaZnAsO eps_inf = 9.4644


In [4]:
# Extract eps_ionic
for root, dirs, files in os.walk('../raw_data/hybrid_calcs_raw/'):
    if root.endswith('dfpt'):
        for name in files:
            if name == 'vasprun.xml':
                vasprun = Vasprun(os.path.join(root,name))
                form = vasprun.structures[0].composition.reduced_formula
                eps_ion = vasprun.epsilon_ionic
                eps_ion = np.linalg.eig(eps_ion)[0]
                eps_ion = np.mean(eps_ion)
                print(form, 'eps_ion = {}'.format(eps_ion))
                for i in data_dicts:
                    if i['formula'] == form:
                        i['eps_ion'] = eps_ion

NaSbO3 eps_ion = 3.9547976433333325
YZnPO eps_ion = 10.790880043333331
NaAg3O2 eps_ion = 1.92703057
NaNbO2 eps_ion = 2.7240350433333336
LiNbO2 eps_ion = 6.3393898533333335
CuRhO2 eps_ion = 1.365788386666667
LaZnAsO eps_ion = 18.191465400000002
LiAg3O2 eps_ion = 2.8225352333333333
PtO2 eps_ion = 0.9732904166666669
Ca4Bi2O eps_ion = 36.55105046
K2TiO3 eps_ion = 5.887969529999999
YZnAsO eps_ion = 21.187880353333327


In [5]:
# Extract effective mass
for root, dirs, files in os.walk('../raw_data/hybrid_calcs_raw/'):
    if root.endswith('loptics/loptics'):
        for name in files:
            if name == 'vasprun.xml':
                vasprun = Vasprun(os.path.join(root,name))
                bs = vasprun.get_band_structure()
                nelect = vasprun.parameters['NELECT']
                concs = [1e+18]
                form = vasprun.structures[0].composition.reduced_formula
                print(form)
                btr = BoltztrapRunner(bs, nelect, doping=concs, 
                                      tmax=300, tgrid=100, energy_grid=0.002)
                btr_dir = btr.run(path_dir=root)
                with open(os.path.join(btr_dir, 'boltztrap.outputtrans'), 'a') as f:
                    for i,x in enumerate (np.concatenate((np.array(concs), -np.array(concs)))):
                        f.write("Doping level number {} n = {} carriers/cm3\n".format(i, x))
            
                bta = BoltztrapAnalyzer.from_files(btr_dir)
                eff_mass_data = bta.get_average_eff_mass(output='eigs', doping_levels=True)
                n_eff_mass = eff_mass_data['n'][300][0]
                print('n:', n_eff_mass)
                p_eff_mass = eff_mass_data['p'][300][0]
                print('p: ', p_eff_mass)
                for i in data_dicts:
                    if i['formula'] == form:
                        i['n_eff_mass'] = n_eff_mass
                        i['p_eff_mass'] = p_eff_mass

NaSbO3
n: [0.3518373994857706, 0.3518379222427252, 0.5596881906707433]
p:  [8.68184422177211, 8.68185732833821, 13.930882583161804]
NaAg3O2
n: [0.6372232742457729, 0.9563897839034763, 0.9804838352168037]
p:  [0.6769420969887537, 0.8030784807522859, 5.1866496131911415]
PtO2
n: [0.39992937393026423, 0.4044809053711993, 1.3925364897553876]
p:  [0.8129081006998773, 2.3181547377284937, 2.8912389541510866]
CuRhO2
n: [0.45866163769016544, 0.45866322036019125, 0.6197820546797479]
p:  [1.7274368836245189, 1.7274429196206877, 4.562151100786986]
LiAg3O2
n: [0.8899812319503068, 0.943972218524671, 1.0772204513027566]
p:  [0.34777219880483856, 0.7026923464430791, 1.988543250823546]
K2TiO3
n: [0.8779510699251204, 2.4248277346365046, 4.782626907876998]
p:  [0.9877743759206171, 3.1184794437114967, 58.22028236381725]
Ca4Bi2O
n: [0.4628720487530518, 0.4639346987074158, 1.1796648875844193]
p:  [0.601331120381792, 0.6035524761171134, 0.7244918217781454]
LiNbO2
n: [2.4205156202670226, 2.4205241447760293, 7.

In [6]:
# Turn into dataframe
calc_data = DataFrame(data_dicts)
cols = ['formula', 'eps_ion', 'eps_inf', 'n_eff_mass', 'p_eff_mass']
calc_data = calc_data[cols]
calc_data.head(12)

,formula,eps_ion,eps_inf,n_eff_mass,p_eff_mass
0,NaSbO3,3.954798,2.784433,"[0.3518373994857706, 0.3518379222427252, 0.559...","[8.68184422177211, 8.68185732833821, 13.930882..."
1,NaAg3O2,1.927031,4.383967,"[0.6372232742457729, 0.9563897839034763, 0.980...","[0.6769420969887537, 0.8030784807522859, 5.186..."
2,PtO2,0.973290,7.086067,"[0.39992937393026423, 0.4044809053711993, 1.39...","[0.8129081006998773, 2.3181547377284937, 2.891..."
3,CuRhO2,1.365788,8.247800,"[0.45866163769016544, 0.45866322036019125, 0.6...","[1.7274368836245189, 1.7274429196206877, 4.562..."
4,LiAg3O2,2.822535,4.962000,"[0.8899812319503068, 0.943972218524671, 1.0772...","[0.34777219880483856, 0.7026923464430791, 1.98..."
5,K2TiO3,5.887970,2.992133,"[0.8779510699251204, 2.4248277346365046, 4.782...","[0.9877743759206171, 3.1184794437114967, 58.22..."
6,Ca4Bi2O,36.551050,9.909700,"[0.4628720487530518, 0.4639346987074158, 1.179...","[0.601331120381792, 0.6035524761171134, 0.7244..."
7,LiNbO2,6.339390,7.705967,"[2.4205156202670226, 2.4205241447760293, 7.207...","[0.4237244529207844, 0.4237259374635281, 1.747..."
8,YZnAsO,21.187880,8.693433,"[0.6512046401717118, 0.6512046401717121, 1.120...","[1.1765934784736358, 1.176593478473636, 1.7686..."
9,NaNbO2,2.724035,7.329100,"[0.9712358255366492, 1.4948749434822537, 1.494...","[0.5038388527611387, 0.5038392761600533, 1.138..."


In [7]:
# Calculate average (polycrystalline) effective mass using harmonic mean
calc_data['n_eff_mass_poly'] = calc_data.apply(lambda x: hmean(x['n_eff_mass']), axis=1)
calc_data['p_eff_mass_poly'] = calc_data.apply(lambda x: hmean(x['p_eff_mass']), axis=1)

# Calculate static dielectric constants
calc_data['eps_total'] = calc_data['eps_inf'] + calc_data['eps_ion']

# Calculate epsilon_effective and E_polaron (in eV)
calc_data['eps_eff'] = 1./ ( (1./ calc_data['eps_inf']) - 
                                (1./ calc_data['eps_total']) )

def E_polaron(eff_mass, eps_eff):
    E =  (eff_mass/(4*(eps_eff**2))) * 13.605698066
    return(E)

calc_data['E_polaron_hole'] = calc_data.apply(lambda x: E_polaron(x['p_eff_mass_poly'],
                                                                       x['eps_eff']), axis=1)
calc_data['E_polaron_electron'] = calc_data.apply(lambda x: E_polaron(x['n_eff_mass_poly'],
                                                                       x['eps_eff']), axis=1)


In [8]:
# Write out to json
with open('../processed_data/hybrid_calcs_data.json', 'w') as f:
    out = calc_data.to_json()
    f.write(out)

**Mobilities**

In [9]:
# Run vasp-ir script in each dfpt directory to get simulated spectrum
for root, dirs, files in os.walk('../raw_data/hybrid_calcs_raw/'):
    if root.endswith('dfpt'):
        subprocess.Popen(['vasp-ir'], cwd=root)

In [9]:
# run characteristic_phonon_freq.jl to get characteristic phonon frequencies
os.system('julia scripts/characteristic_phonon_freq.jl');

In [10]:
# Read in generated phonon frequencies
phonon_freqs = pd.read_csv('../processed_data/characteristic_frequencies.csv', header=None)
# Tidy dataframe 
phonon_freqs = phonon_freqs.rename(columns={0: 'directory', 1:'omega'})
phonon_freqs.head(12)

,directory,omega
0,Ca4Bi2O_dfpt,105.368650
1,CuRhO2_dfpt,590.144263
2,K2TiO3_dfpt,370.013960
3,LaZnOAs_dfpt,281.453383
4,LiAg3O2_dfpt,428.787160
5,LiNbO2_dfpt,419.763007
6,NaAg3O2_dfpt,439.473386
7,NaNbO2_dfpt,464.000723
8,NaSbO3_dfpt,452.817921
9,PtO2_dfpt,712.924892


In [11]:
# Add omega column to data
# MIND: This is not done using a proper merge because the directory names differ from 
# formulas generated automatically using pymatgen. ensure the order is consistent.
calc_data['omega'] = phonon_freqs['omega']

In [12]:
calc_data.head(12)

,formula,eps_ion,eps_inf,n_eff_mass,p_eff_mass,n_eff_mass_poly,p_eff_mass_poly,eps_total,eps_eff,E_polaron_hole,E_polaron_electron,omega
0,NaSbO3,3.954798,2.784433,"[0.3518373994857706, 0.3518379222427252, 0.559...","[8.68184422177211, 8.68185732833821, 13.930882...",0.401545,9.928890,6.739231,4.744854,1.500085,0.060666,105.368650
1,NaAg3O2,1.927031,4.383967,"[0.6372232742457729, 0.9563897839034763, 0.980...","[0.6769420969887537, 0.8030784807522859, 5.186...",0.825352,1.029074,6.310997,14.357427,0.016981,0.013619,590.144263
2,PtO2,0.973290,7.086067,"[0.39992937393026423, 0.4044809053711993, 1.39...","[0.8129081006998773, 2.3181547377284937, 2.891...",0.527161,1.494469,8.059357,58.676363,0.001476,0.000521,370.013960
3,CuRhO2,1.365788,8.247800,"[0.45866163769016544, 0.45866322036019125, 0.6...","[1.7274368836245189, 1.7274429196206877, 4.562...",0.502178,2.178685,9.613588,58.055080,0.002199,0.000507,281.453383
4,LiAg3O2,2.822535,4.962000,"[0.8899812319503068, 0.943972218524671, 1.0772...","[0.34777219880483856, 0.7026923464430791, 1.98...",0.964231,0.624815,7.784535,13.685166,0.011348,0.017512,428.787160
5,K2TiO3,5.887970,2.992133,"[0.8779510699251204, 2.4248277346365046, 4.782...","[0.9877743759206171, 3.1184794437114967, 58.22...",1.704055,2.221857,8.880103,4.512668,0.371116,0.284628,419.763007
6,Ca4Bi2O,36.551050,9.909700,"[0.4628720487530518, 0.4639346987074158, 1.179...","[0.601331120381792, 0.6035524761171134, 0.7244...",0.580990,0.638283,46.460750,12.596412,0.013683,0.012455,439.473386
7,LiNbO2,6.339390,7.705967,"[2.4205156202670226, 2.4205241447760293, 7.207...","[0.4237244529207844, 0.4237259374635281, 1.747...",3.108767,0.566874,14.045357,17.073102,0.006615,0.036276,464.000723
8,YZnAsO,21.187880,8.693433,"[0.6512046401717118, 0.6512046401717121, 1.120...","[1.1765934784736358, 1.176593478473636, 1.7686...",0.756921,1.324366,29.881314,12.260368,0.029968,0.017128,452.817921
9,NaNbO2,2.724035,7.329100,"[0.9712358255366492, 1.4948749434822537, 1.494...","[0.5038388527611387, 0.5038392761600533, 1.138...",1.267149,0.618844,10.053135,27.048269,0.002877,0.005891,712.924892


In [13]:
# Export simple data file to be read in by julia script to calculate mobilities
export_data = calc_data[['eps_inf', 'eps_total', 'omega', 
                         'n_eff_mass_poly', 'p_eff_mass_poly']]
export_data.to_csv('../processed_data/mobility_input.csv', index=False)

In [16]:
# run mobility_calculation.jl to get electron and hole mobilities
os.system('julia scripts/mobility_calculation.jl')

0

In [14]:
# Read in calculated mobilities
mobility_data = pd.read_csv('../processed_data/mobility_output.csv', header=None)
mobility_data = mobility_data.rename(columns={0: 'n_mobility', 1:'p_mobility'})
mobility_data.head(12)

,n_mobility,p_mobility
0,30.589132,26.200116
1,631.025651,67.331576
2,1.293661,0.697609
3,67.437849,16.570584
4,29.734437,59.856369
5,5.568217,88.428108
6,41.754521,29.233625
7,47.453522,144.692869
8,32.633851,0.062644
9,853.448328,175.066563


In [15]:
# Add mobility columns to data
# MIND: This is not done using a proper merge again. Check the order.
calc_data['n_mobility'] = mobility_data['n_mobility']
calc_data['p_mobility'] = mobility_data['p_mobility']
calc_data.head(12)

,formula,eps_ion,eps_inf,n_eff_mass,p_eff_mass,n_eff_mass_poly,p_eff_mass_poly,eps_total,eps_eff,E_polaron_hole,E_polaron_electron,omega,n_mobility,p_mobility
0,NaSbO3,3.954798,2.784433,"[0.3518373994857706, 0.3518379222427252, 0.559...","[8.68184422177211, 8.68185732833821, 13.930882...",0.401545,9.928890,6.739231,4.744854,1.500085,0.060666,105.368650,30.589132,26.200116
1,NaAg3O2,1.927031,4.383967,"[0.6372232742457729, 0.9563897839034763, 0.980...","[0.6769420969887537, 0.8030784807522859, 5.186...",0.825352,1.029074,6.310997,14.357427,0.016981,0.013619,590.144263,631.025651,67.331576
2,PtO2,0.973290,7.086067,"[0.39992937393026423, 0.4044809053711993, 1.39...","[0.8129081006998773, 2.3181547377284937, 2.891...",0.527161,1.494469,8.059357,58.676363,0.001476,0.000521,370.013960,1.293661,0.697609
3,CuRhO2,1.365788,8.247800,"[0.45866163769016544, 0.45866322036019125, 0.6...","[1.7274368836245189, 1.7274429196206877, 4.562...",0.502178,2.178685,9.613588,58.055080,0.002199,0.000507,281.453383,67.437849,16.570584
4,LiAg3O2,2.822535,4.962000,"[0.8899812319503068, 0.943972218524671, 1.0772...","[0.34777219880483856, 0.7026923464430791, 1.98...",0.964231,0.624815,7.784535,13.685166,0.011348,0.017512,428.787160,29.734437,59.856369
5,K2TiO3,5.887970,2.992133,"[0.8779510699251204, 2.4248277346365046, 4.782...","[0.9877743759206171, 3.1184794437114967, 58.22...",1.704055,2.221857,8.880103,4.512668,0.371116,0.284628,419.763007,5.568217,88.428108
6,Ca4Bi2O,36.551050,9.909700,"[0.4628720487530518, 0.4639346987074158, 1.179...","[0.601331120381792, 0.6035524761171134, 0.7244...",0.580990,0.638283,46.460750,12.596412,0.013683,0.012455,439.473386,41.754521,29.233625
7,LiNbO2,6.339390,7.705967,"[2.4205156202670226, 2.4205241447760293, 7.207...","[0.4237244529207844, 0.4237259374635281, 1.747...",3.108767,0.566874,14.045357,17.073102,0.006615,0.036276,464.000723,47.453522,144.692869
8,YZnAsO,21.187880,8.693433,"[0.6512046401717118, 0.6512046401717121, 1.120...","[1.1765934784736358, 1.176593478473636, 1.7686...",0.756921,1.324366,29.881314,12.260368,0.029968,0.017128,452.817921,32.633851,0.062644
9,NaNbO2,2.724035,7.329100,"[0.9712358255366492, 1.4948749434822537, 1.494...","[0.5038388527611387, 0.5038392761600533, 1.138...",1.267149,0.618844,10.053135,27.048269,0.002877,0.005891,712.924892,853.448328,175.066563


In [16]:
# Export dataframe 
# Write out to json
with open('../processed_data/hybrid_calcs_data_and_mobilities.json', 'w') as f:
    out = calc_data.to_json()
    f.write(out)